In [1]:
import pandas as pd

## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

+ если источник traffic_source равен yandex или google, то в source_type ставится organic
+ для источников paid и email из России - ставим ad
+ для источников paid и email не из России - ставим other

все остальные варианты берем из traffic_source без изменений

In [80]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head(1)

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex


In [81]:
log['traffic_source'].value_counts()

direct    7013
email     3875
yandex    2900
paid      2643
google    2507
Name: traffic_source, dtype: int64

In [82]:
def source_classification(row):
    
    if row['traffic_source'] == 'yandex' or row['traffic_source'] == 'google':
        return 'organic'
    elif row['traffic_source'] in ('paid', 'email') and row['region'] == 'Russia':
        return 'ad'
    elif row['traffic_source'] in ('paid', 'email') and row['region'] != 'Russia':
        return 'other'
    
    return row['traffic_source']

In [83]:
log['source_type'] = log.apply(source_classification, axis=1)

In [84]:
log['source_type'].value_counts()

direct     7013
organic    5407
ad         3790
other      2728
Name: source_type, dtype: int64

## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. 

Выполните следующие действия:

Прочитайте содержимое файла в датафрейм

Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [106]:
import re

In [160]:
data = []
with open ('URLs.txt', "r") as myfile:
    for line in myfile:
        data.append(line)

myfile.close()
        
df = pd.DataFrame(data)

In [161]:
df.columns = ['address']

In [162]:
df.head(1)

,address
0,url\n


In [164]:
df[ df.address.str.contains('/' + r'\d{8}' + '-') ].head()

,address
4,/politics/36188461-s-marta-zhizn-rossiyan-susc...
5,/world/36007585-tramp-pridumal-kak-reshit-ukra...
6,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
7,/video/36001498-poyavilis-pervye-podrobnosti-g...
8,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [165]:
len(df)- len(df[ df.address.str.contains('/' + r'\d{8}' + '-') ])

10

## Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [5]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
grouped_users = ratings.groupby('userId').count().reset_index()
filtered_users = grouped_users[ (grouped_users['movieId'] > 99) ]

users_list = filtered_users['userId'].tolist()

filtered_ratings = ratings [ ratings['userId'].isin(users_list) ]
first_date = filtered_ratings[['userId', 'timestamp']].groupby('userId').min().reset_index()
last_date = filtered_ratings[['userId', 'timestamp']].groupby('userId').max().reset_index()

first_date.rename(columns={'timestamp': 'timestamp_first'}, inplace=True)
last_date.rename(columns={'timestamp': 'timestamp_last'}, inplace=True)

joined_table = first_date.merge(last_date)
joined_table['lifetime, days'] = round ((joined_table['timestamp_last'] - joined_table['timestamp_first']) /60 /60 /24 , 1)
joined_table.sort_values('lifetime, days', ascending = False).head()

,userId,timestamp_first,timestamp_last,"lifetime, days"
140,380,949367125,1465156786,5969.8
212,547,974777109,1476587644,5808.0
161,427,939332587,1415901843,5515.8
3,15,997937239,1469330735,5455.9
248,624,1019123866,1476616373,5295.1


## Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).

Необходимо сформировать две таблицы:
+ таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
+ аналогичную таблицу по типам выручки с указанием адреса клиента

In [8]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd.head(1)

,client_id,rzd_revenue
0,111,1093


In [9]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto.head(1)

,client_id,auto_revenue
0,113,57483


In [10]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air.head(1)

,client_id,air_revenue
0,115,81


In [12]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base.head(1)

,client_id,address
0,111,Комсомольская 4


### делаем  таблицы:

In [13]:
revenue_by_clients = rzd.merge(auto, how = 'outer').merge(air, how = 'outer')
revenue_by_clients

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [14]:
revenue_by_clients_addresses = client_base.merge(revenue_by_clients, how = 'outer')
revenue_by_clients_addresses

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0
